## cnews文本数据矩阵化成词袋

In [23]:
import numpy as np
import pandas as pd
import jieba

### 1、先读取数据并查看

In [3]:
train_data = pd.read_csv('../dataset/cnews/cnews.train.txt',sep='\t',engine='python',encoding='UTF-8',names=['label','content'])
print(train_data.head())

  label                                            content
0    体育  马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1    体育  商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2    体育  冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3    体育  辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4    体育  揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道  谢亚龙已经被公安...


### 2、看一下停用词读取

In [20]:
# 加载停用词列表
def get_stopwords(path):
    stopwords = open(path,encoding='utf8').read().strip().split('\n')
    return stopwords

stopwords = get_stopwords('./stopwords.txt')
print(stopwords)

['，', '。', '/', ' ', ' ', '！', '■', '#', '{', '}', '（', '）', '[', ']', '【', '】', '“', '”', '+', '-', '_', '(', ')', '~', '`', '*', '－', '—', '～', '•', '●', '⊙', '=', '!', '□', '／', '@', '：', '？', '?', '￥', '$', '、', '‘', '’', "'", ';', ',', '.', ':', ';', '"', '；', '的', '了', '呢', '么', '啊', '们', '你', '我', '它', '他', '她', '你们', '我们', '他们', '她们', '它们', '这', '那', '能', '可', '也', '来', '让', '又', '下', '上', '中', '在', '《', '》', '…', '·', '吧', '吗']


In [5]:
# 将文本去除停用词
def move_stopwords(text, stopwords):
    return [i for i in text if i not in stopwords]

In [6]:
# 将一篇文档转成分词列表形式(并去除停用词)
def doc2wordlist(text):
    words = jieba.lcut(text)
    stopwords = get_stopwords('./stopwords.txt')
    return move_stopwords(words,stopwords)

# tf = train_data.content.apply(doc2wordlist)
# tf.head()

In [7]:
from collections import Counter
# 构建词典，默认取频数最大的前5000词
def build_dict(texts,path,max_size=5000):
    alldata = []
    for i in texts:
        alldata.extend(i)
    
    counter = Counter(alldata)
    
    wordslist = []
    for item in counter.most_common(max_size):
        wordslist.append(item[0])

    with open(path,'w',encoding='utf-8') as f:
        s = '\n'.join(wordslist)
        f.write(s)
    
    return

In [8]:
# 获取字典，{词语：序号}
def get_dict(path):
    vocab = {}
    with open(path,'r',encoding='utf-8') as f:
        for i,word in enumerate(f):
            vocab[word.strip()] = i
    
    return vocab

In [9]:
# 将一篇文档词列表 转换 成词袋向量
def text2bag(text,vocab):
    vec = [0] * len(vocab)

    for word in text:
        try:
            vec[vocab[word]] += 1
        except (KeyError,ValueError):
            continue
    return vec

In [ ]:
# 处理，将所有文档的内容处理成词列表，并去除停用词
words_list = []
for text in train_data.content:
    words_list.append(doc2wordlist(text))

In [21]:
# 建立字典
build_dict(words_list,'./vocab.txt')
# 获取词典{单词：序号}
vocab = get_dict('./vocab.txt')

for i,item in enumerate(vocab):
    print(item)
    if i>10 :
        break

是
和
有
都

将
月
就
对
年
为
一个


In [22]:
# 构建词袋X
bow = []
for i in words_list:
    bow.append(text2bag(i,vocab))

bow = np.array(bow)
print(bow.shape,bow,sep='\n')

(50000, 5000)
[[2 0 2 ... 0 0 0]
 [4 3 7 ... 0 0 0]
 [2 4 2 ... 0 0 0]
 ...
 [0 2 3 ... 0 0 0]
 [0 1 2 ... 0 0 0]
 [3 7 1 ... 0 0 0]]


## 使用sklearn包工具获取词袋，词频，TF-IDF 

In [44]:
import sklearn.feature_extraction.text as ft

# 构建计数适量化器
cv = ft.CountVectorizer()
# 获取词袋矩阵
bow = cv.fit_transform([' '.join(x) for x in words_list]) #words_list 是已经分词且去停用词的词列表
# print(bow)
print(bow.shape,sep='\n')

(50000, 331413)


In [43]:
# 词频 TF
from sklearn import preprocessing
tf = preprocessing.normalize(bow,norm='l1')
# print(tf)
print(tf.shape)

(50000, 331413)


In [46]:
# 构建词频逆文档频率转换器
tfidf_model = ft.TfidfTransformer()
# TF-IDF
tfidf = tfidf_model.fit_transform(bow)
# print(tfidf)
print(tfidf.shape)

(50000, 331413)
